**Image Processing Review** <br>
Author: Melissa

This NB uses the file archive_image_list.txt, which was from all images for 2021A-0113 in the NOIRLab Data Archive.

This NB does the following:

1. put select columns from exposures and subtractions tables into numpy arrays
2. read in the archive list of all images into numpy arrays
3. check how many archive images have associated exposures
   * DECaPS images from archive are not yet in the exposures table (known)
4. check how many exposures have associated subtractions
   * COSMOS i-band do not yet have subtractions
5. plot distributions of the image characteristics
   * **only for 'standard' image exposure times** (not the -AGN or the -MOON images)
   * limiting magnitude, zeropoint, seeing, and "skysig" (standard dev of sky pixels??)
   * also plot lim mag, seeing, and skysig vs. zeropoint
6. plot distributions of the image characteristics
   * for all the images, regardless of exposure time
   * can see that discrepant mag_zp values aren't due to being near the moon

<br>
Exposure times by "class" or type of image; only some are "standard" for the DDF.<br>

`| Image Class  | filters | exposure times |     |`<br>
`| COSMOS       | g r i   | 60  86  130    | std |`<br>
`| COSMOS MOON  | g r i   | 60  30  30     |     |`<br>
`| COSMOS AGN   | g r i   | 80  70  90     |     |`<br>
`| ELAIS        | g r i   | 60  86  130    | std |`<br>
`| DECaPS       | g r i z | 96  50  30  30 | std |`<br>
`| DECaPS MOON  | g r i z | -   25  30  30 |     |`<br>


### 0. Set Up

In [ ]:
import psycopg2
import psycopg2.extras
import getpass

import os
import numpy as np
import matplotlib.pyplot as plt
from copy import deepcopy

In [ ]:
dbuser = input("DB User: ")
dbpasswd = getpass.getpass("DB Password: ")

In [ ]:
db = psycopg2.connect(f"dbname='decat' user='{dbuser}' password='{dbpasswd}' host='decatdb.lbl.gov'")

In [ ]:
cursor = db.cursor( cursor_factory = psycopg2.extras.DictCursor )

In [ ]:
### If you want to print table schema
# tables = ['exposures','subtractions']
# for table in tables:
#     query = "SELECT column_name, data_type FROM information_schema.columns WHERE table_name=%s"
#     cursor.execute( query, ( table, ))
#     print( f"\nTABLE: {table}\n===========================" )
#     for row in cursor:
#         print( f"{row['column_name']:24s}  :  {row['data_type']:s}" )

### 1.1 exposures table: put selected columns into numpy arrays

In [ ]:
query = "SELECT id, ra, dec, filename, mjd, filter, proposalid, header FROM exposures"
cursor.execute( query )
results = np.array( cursor.fetchall() ).transpose()
exp_id  = np.asarray( results[0], dtype='int' )
exp_ra  = np.asarray( results[1], dtype='float' )
exp_dec = np.asarray( results[2], dtype='float' )
exp_fnm = np.asarray( results[3], dtype='str' )
exp_mjd = np.asarray( results[4], dtype='float' )
exp_fil = np.asarray( results[5], dtype='str' )
exp_pid = np.asarray( results[6], dtype='str' )
exp_hdr = np.asarray( results[7], dtype='str' )
del query, results

In [ ]:
### exp_fbs : just the filename base (without the extension)
exp_fbs = deepcopy(exp_fnm)
for i,fnm in enumerate(exp_fnm):
    exp_fbs[i] = fnm.split('.')[0]

print(exp_fnm[0])
print(exp_fbs[0])

In [ ]:
### The header is a string that can be converted to a directory
# temp = eval(exp_hdr[0])
# temp
# print("temp['AZ'] = ",temp['AZ'])

### 1.2 subtractions table: put selected columns into numpy arrays

In [ ]:
query = "SELECT id, ra, dec, lmt_mg, seeing, skysig, magzp, "+ \
        "ccdnum, image_id, exposure_id FROM subtractions"
cursor.execute( query )
results = np.array( cursor.fetchall() ).transpose()
sub_id  = np.asarray( results[0], dtype='int' )
sub_ra  = np.asarray( results[1], dtype='float' )
sub_dec = np.asarray( results[2], dtype='float' )
sub_lmg = np.asarray( results[3], dtype='float' )
sub_see = np.asarray( results[4], dtype='float' )
sub_sks = np.asarray( results[5], dtype='float' )
sub_mzp = np.asarray( results[6], dtype='float' )
sub_ccd = np.asarray( results[7], dtype='int' )
sub_iid = np.asarray( results[8], dtype='int' )
sub_eid = np.asarray( results[9], dtype='int' )
del query, results

In [ ]:
### sub_fil : the filter of the exposure associated with the subtraction
### sub_mjd : the MJD of the exposure associated with the subtraction
sub_fil = np.zeros( len(sub_eid), dtype='str' )
sub_mjd = np.zeros( len(sub_eid), dtype='float' )
for i in range(len(sub_eid)):
    tx = np.where( exp_id == sub_eid[i] )[0]
    if len(tx) == 1:
        sub_fil[i] = exp_fil[tx[0]]
        sub_mjd[i] = exp_mjd[tx[0]]
    elif len(tx) == 0:
        print('No exposure id match for: ', sub_eid[i])
    elif len(tx) > 1:
        print('More than one exposure id match for: ', sub_eid[i])
# print(sub_fil)

In [ ]:
### Print value ranges for the exposure id, subtraction id, subtraction ccd
# print(np.min(exp_id),np.max(exp_id))
# print(np.min(sub_ccd),np.max(sub_ccd))
# print(np.min(sub_iid),np.max(sub_iid))
# print(np.min(sub_eid),np.max(sub_eid))

In [ ]:
### Are there any bad ccds?
### Yes. CCD = 31, 61 have no subtractions
### Known issue: http://www.ctio.noao.edu/noao/node/2630

# usebins = np.arange( 63, dtype='float' ) + 0.5
# plt.hist(sub_ccd, bins=usebins, histtype='step')
# plt.xlabel('CCD Number')
# plt.ylabel('Number of Subtractions')

### Print number of subtractions per CCD as a check
# for i in range(63):
#     tx = np.where( sub_ccd == i )[0]
#     print(i, len(tx))
#     del tx

In [ ]:
### Show map of RA, Dec of the COSMOS and ELAIS subtractions
# plt.figure(figsize=(12, 4))
# plt.subplot(1, 2, 1)
# cx = np.where( (sub_ra>145) & (sub_ra<155) )[0]
# plt.plot( sub_ra[cx], sub_dec[cx], 'o', ms=10, alpha=0.01, color='dodgerblue')
# plt.subplot(1, 2, 2)
# ex = np.where( (sub_ra>2.0) & (sub_ra<12.0) )[0]
# plt.plot( sub_ra[ex], sub_dec[ex], 'o', ms=10, alpha=0.01, color='darkviolet')

### 2. archive image list: read it into numpy arrays
Melissa made this list of all 2021A-0113 images from the NOIRLab Data Archive.

* fbase : base of archive filename (without extension)
* ra, dec : coordinates in decimal degrees
* mjd : modified julian date of observation (at exposure start)
* filt : filter
* airm : airmass
* obj : object name
* expt : exposure time
* seqid : sequence id (relates to JSON file for observing)
* class : COSMOS, COSMOS-AGN, COSMOS-MOON, ELAIS, DECaPS, DECaPS-MOON, or NONE
   * -AGN and -MOON indicate non-standard exposure times
* ftype : field type, EGAL, GAL, or NONE
* moonsep : moon separation in degrees
* moonill : moon illumination as a fraction
* mpval : median image pixel value for images with moonsep<60 and moonill>0.2

In [ ]:
fnm = 'archive_image_list.txt'
arch_fbase   = np.loadtxt( fnm, dtype='str', usecols=(0) )
arch_ra      = np.loadtxt( fnm, dtype='float', usecols=(1) )
arch_dec     = np.loadtxt( fnm, dtype='float', usecols=(2) )
arch_mjd     = np.loadtxt( fnm, dtype='float', usecols=(3) )
arch_filt    = np.loadtxt( fnm, dtype='str', usecols=(4) )
arch_airm    = np.loadtxt( fnm, dtype='float', usecols=(5) )
arch_obj     = np.loadtxt( fnm, dtype='str', usecols=(6) )
arch_expt    = np.loadtxt( fnm, dtype='float', usecols=(7) )
arch_seqid   = np.loadtxt( fnm, dtype='str', usecols=(8) )
arch_class   = np.loadtxt( fnm, dtype='str', usecols=(9) )
arch_ftype   = np.loadtxt( fnm, dtype='str', usecols=(10) )
arch_moonsep = np.loadtxt( fnm, dtype='float', usecols=(11) )
arch_moonill = np.loadtxt( fnm, dtype='float', usecols=(12) )
arch_mpval   = np.loadtxt( fnm, dtype='float', usecols=(13) )

#### 3.1 match archive image list to the exposures table

In [ ]:
### exp_aid : the index of the exposure in the archive table
### exp_std : if the archive image has a "standard" classification (COSMOS, ELAIS, or DECaPS only)
exp_aid = np.zeros( len(exp_id), dtype='int' )
exp_std = np.zeros( len(exp_id), dtype='int' )

### arch_nem : number of exposure table matches
### arch_eid : expsoure id of match (first, if multiple)
arch_nem = np.zeros( len(arch_fbase), dtype='int' )
arch_eid = np.zeros( len(arch_fbase), dtype='int' )

for i,fbase in enumerate(arch_fbase):
    tx = np.where( fbase == exp_fbs )[0]
    arch_nem[i] = len(tx)
    if len(tx) == 0:
        arch_eid[i] = -1
    elif len(tx) >= 1:
        arch_eid[i] = exp_id[tx[0]]
        exp_aid[tx[0]] = i
        if (arch_class[i] == 'COSMOS') | \
        (arch_class[i] == 'ELAIS') | \
        (arch_class[i] == 'DECaPS'):
            exp_std[tx[0]] = 1
    del tx

print('Number of archive images with N matches to exposure table:')
for i in range(4):
    tx = np.where( arch_nem == i )[0]
    print(i, len(tx))
    del tx
print('')
print('Classes of archive images without an exposure table match:')
tx = np.where( arch_nem == 0 )[0]
tmps = np.asarray( arch_class[tx], dtype='str')
utmps = np.unique( np.sort( tmps ) )
for u,utmp in enumerate(utmps):
    tx = np.where( tmps == utmp )[0]
    print(utmp, len(tx))
    del tx

del tmps,utmps

#### 3.2 add a few more arrays to the sub_ and exp_ sets

In [ ]:
### sub_expt    : exposure time of the image for this subtraction
### sub_moonsep : moon separation in degrees of the image for this subtraction
### sub_moonill : moon illumination (fraction) of the image for this subtraction
### sub_std     : is a standard image (COSMOS, ELAIS, or DECapS; no -AGN, no -MOON)

sub_expt    = np.zeros( len(sub_eid), dtype='float' )
sub_moonsep = np.zeros( len(sub_eid), dtype='float' )
sub_moonill = np.zeros( len(sub_eid), dtype='float' )
sub_mpixval = np.zeros( len(sub_eid), dtype='float' )
sub_std     = np.zeros( len(sub_eid), dtype='int' )
for i in range(len(sub_eid)):
    tx = np.where( exp_id == sub_eid[i] )[0]
    archid = exp_aid[tx[0]]
    sub_expt[i] = arch_expt[archid]
    sub_moonsep[i] = arch_moonsep[archid]
    sub_moonill[i] = arch_moonill[archid]
    sub_mpixval[i] = arch_mpval[archid]
    sub_std[i] = exp_std[tx[0]]

### 4. Which exposures do not have subtractions, and why.

In [ ]:
### Number of subtractions per exposure
exp_nsubs = np.zeros( len(exp_id), dtype='int' )
for i,expid in enumerate(exp_id):
    tx = np.where( sub_eid == expid )[0]
    exp_nsubs[i] = len(tx)
    del tx

print('Number of subtractions; number of exposures with N subtractions.')
tmps = np.sort(exp_nsubs)
utmps = np.unique(tmps)
for u,utmp in enumerate(utmps):
    tx = np.where( tmps == utmp )[0]
    print(utmp,len(tx))
    del tx

In [ ]:
ex = np.where( exp_nsubs == 0 )[0]

print('class, number of exposures without subtractions')
nosub_class = arch_class[ exp_aid[ex] ]
tmps = np.sort( nosub_class )
utmps = np.unique( tmps )
for u,utmp in enumerate(utmps):
    tx = np.where( tmps == utmp )[0]
    print(utmp,len(tx))
    del tx
    
print('')
print('filter, number of exposures without subtractions')
nosub_fil = exp_fil[ex]
tmps = np.sort( nosub_fil )
utmps = np.unique( tmps )
for u,utmp in enumerate(utmps):
    tx = np.where( tmps == utmp )[0]
    print(utmp,len(tx))
    del tx

print('')
print('The exposures without subtractions are primarily the COSMOS i-band')

In [ ]:
### Double check on ELAIS processing
# x = np.where( arch_class == 'ELAIS' )[0]
# for i in x:
#     # find matching exposure to this filename base
#     tx = np.where( arch_fbase[i] == exp_fbs )[0]
#     # find all matching subtractions for the exposure
#     sx = np.where( sub_eid == exp_id[tx] )[0]
#     if len(tx) == 1:
#         print(i,arch_class[i],exp_fbs[tx],len(sx))

### 5. Plot distributions of the image characteristics

Only for subtractions of 'standard' exposures/images, that have a class of COSMOS, ELAIS (or DECaPS, if there were any).
This removes the -AGN and -MOON exposures which have different (shorter) exposure times.

`| COSMOS       | g r i   | 60  86  130    | std |`<br>
`| COSMOS MOON  | g r i   | 60  30  30     |     |`<br>
`| COSMOS AGN   | g r i   | 80  70  90     |     |`<br>
`| ELAIS        | g r i   | 60  86  130    | std |`<br>
`| DECaPS       | g r i z | 96  50  30  30 | std |`<br>
`| DECaPS MOON  | g r i z | -   25  30  30 |     |`<br>

I.e., only use subtractions with `sub_std == 1`.

In [ ]:
### COSMOS subtractions in g and r with standard exposure times
cxg = np.where( (sub_ra>145) & (sub_ra<155)  & (sub_fil == 'g') & (sub_std == 1) )[0]
cxr = np.where( (sub_ra>145) & (sub_ra<155)  & (sub_fil == 'r') & (sub_std == 1) )[0]

### ELAIS subtractions in g and r with standard exposure times
exg = np.where( (sub_ra>2.0) & (sub_ra<12.0) & (sub_fil == 'g') & (sub_std == 1) )[0]
exr = np.where( (sub_ra>2.0) & (sub_ra<12.0) & (sub_fil == 'r') & (sub_std == 1) )[0]
exi = np.where( (sub_ra>2.0) & (sub_ra<12.0) & (sub_fil == 'i') & (sub_std == 1) )[0]

In [ ]:
plt.figure(figsize=(12, 3))
plt.subplot(1, 3, 1)
plt.hist( [sub_lmg[cxg],sub_lmg[exg]], bins=30, stacked=True, color=['darkgreen','limegreen'], label=['COSMOS','ELAIS'])
plt.xlabel('g-band lmt_mg')
plt.ylabel('number of subtractions')
plt.legend(loc='upper left')
plt.subplot(1, 3, 2)
plt.hist( [sub_lmg[cxr],sub_lmg[exr]], bins=30, stacked=True, color=['red','darkorange'], label=['COSMOS','ELAIS'])
plt.xlabel('r-band lmt_mg')
plt.legend(loc='upper left')
plt.subplot(1, 3, 3)
plt.hist( sub_lmg[exi], bins=30, stacked=True, color='brown', label='ELAIS')
plt.xlabel('i-band lmt_mg')
plt.legend(loc='upper left')
plt.show()

plt.figure(figsize=(12, 3))
plt.subplot(1, 3, 1)
plt.hist( [sub_see[cxg],sub_see[exg]], bins=30, stacked=True, color=['darkgreen','limegreen'], label=['COSMOS','ELAIS'])
plt.xlabel('g-band seeing')
plt.ylabel('number of subtractions')
plt.legend(loc='upper right')
plt.subplot(1, 3, 2)
plt.hist( [sub_see[cxr],sub_see[exr]], bins=30, stacked=True, color=['red','darkorange'], label=['COSMOS','ELAIS'])
plt.xlabel('r-band seeing')
plt.legend(loc='upper right')
plt.subplot(1, 3, 3)
plt.hist( sub_see[exi], bins=30, stacked=True, color='brown', label='ELAIS')
plt.xlabel('i-band seeing')
plt.legend(loc='upper right')
plt.show()

plt.figure(figsize=(12, 3))
plt.subplot(1, 3, 1)
plt.hist( [sub_sks[cxg],sub_sks[exg]], bins=30, stacked=True, color=['darkgreen','limegreen'], label=['COSMOS','ELAIS'])
plt.xlabel('g-band skysig')
plt.ylabel('number of subtractions')
plt.legend(loc='upper right')
plt.subplot(1, 3, 2)
plt.hist( [sub_sks[cxr],sub_sks[exr]], bins=30, stacked=True, color=['red','darkorange'], label=['COSMOS','ELAIS'])
plt.xlabel('r-band skysig')
plt.legend(loc='upper right')
plt.subplot(1, 3, 3)
plt.hist( sub_sks[exi], bins=30, stacked=True, color='brown', label='ELAIS')
plt.xlabel('i-band skysig')
plt.legend(loc='upper right')
plt.show()

plt.figure(figsize=(12, 3))
plt.subplot(1, 3, 1)
plt.hist( [sub_mzp[cxg],sub_mzp[exg]], bins=30, stacked=True, color=['darkgreen','limegreen'], label=['COSMOS','ELAIS'])
plt.xlabel('g-band magzp')
plt.ylabel('number of subtractions')
plt.legend(loc='upper left')
plt.subplot(1, 3, 2)
plt.hist( [sub_mzp[cxr],sub_mzp[exr]], bins=30, stacked=True, color=['red','darkorange'], label=['COSMOS','ELAIS'])
plt.xlabel('r-band magzp')
plt.legend(loc='upper left')
plt.subplot(1, 3, 3)
plt.hist( sub_mzp[exi], bins=30, stacked=True, color='brown', label='ELAIS')
plt.xlabel('i-band magzp')
plt.legend(loc='upper left')
plt.show()

#### 5.2 Plot the zeropoint versus the limiting magnitude, seeing, and sky std dev
For the same subtraction subsets as above

In [ ]:
plt.figure(figsize=(12, 3))
plt.subplot(1, 3, 1)
plt.plot( sub_lmg[cxg], sub_mzp[cxg], 'o', ms=2, alpha=0.1, color='darkgreen', label='COSMOS')
plt.plot( sub_lmg[exg], sub_mzp[exg], 'o', ms=2, alpha=0.1, color='limegreen', label='ELAIS')
plt.xlabel('g-band lmt_mg')
plt.ylabel('g-band mag_zp')
# plt.legend(loc='upper left')
plt.subplot(1, 3, 2)
plt.plot( sub_lmg[cxr], sub_mzp[cxr], 'o', ms=2, alpha=0.1, color='red', label='COSMOS')
plt.plot( sub_lmg[exr], sub_mzp[exr], 'o', ms=2, alpha=0.1, color='darkorange', label='ELAIS')
plt.xlabel('r-band lmt_mg')
plt.ylabel('r-band mag_zp')
# plt.legend(loc='upper left')
plt.subplot(1, 3, 3)
plt.plot( sub_lmg[exi], sub_mzp[exi], 'o', ms=2, alpha=0.1, color='brown', label='ELAIS')
plt.xlabel('i-band lmt_mg')
plt.ylabel('i-band mag_zp')
plt.show()

plt.figure(figsize=(12, 3))
plt.subplot(1, 3, 1)
plt.plot( sub_see[cxg], sub_mzp[cxg], 'o', ms=2, alpha=0.1, color='darkgreen', label='COSMOS')
plt.plot( sub_see[exg], sub_mzp[exg], 'o', ms=2, alpha=0.1, color='limegreen', label='ELAIS')
plt.xlabel('g-band seeing')
plt.ylabel('g-band mag_zp')
# plt.legend(loc='upper left')
plt.subplot(1, 3, 2)
plt.plot( sub_see[cxr], sub_mzp[cxr], 'o', ms=2, alpha=0.1, color='red', label='COSMOS')
plt.plot( sub_see[exr], sub_mzp[exr], 'o', ms=2, alpha=0.1, color='darkorange', label='ELAIS')
plt.xlabel('r-band seeing')
plt.ylabel('r-band mag_zp')
# plt.legend(loc='upper left')
plt.subplot(1, 3, 3)
plt.plot( sub_see[exi], sub_mzp[exi], 'o', ms=2, alpha=0.1, color='brown', label='ELAIS')
plt.xlabel('i-band seeing')
plt.ylabel('i-band mag_zp')
plt.show()

plt.figure(figsize=(12, 3))
plt.subplot(1, 3, 1)
plt.plot( sub_sks[cxg], sub_mzp[cxg], 'o', ms=2, alpha=0.1, color='darkgreen', label='COSMOS')
plt.plot( sub_sks[exg], sub_mzp[exg], 'o', ms=2, alpha=0.1, color='limegreen', label='ELAIS')
plt.xlabel('g-band skysig')
plt.ylabel('g-band mag_zp')
# plt.legend(loc='upper left')
plt.subplot(1, 3, 2)
plt.plot( sub_sks[cxr], sub_mzp[cxr], 'o', ms=2, alpha=0.1, color='red', label='COSMOS')
plt.plot( sub_sks[exr], sub_mzp[exr], 'o', ms=2, alpha=0.1, color='darkorange', label='ELAIS')
plt.xlabel('r-band skysig')
plt.ylabel('r-band mag_zp')
# plt.legend(loc='upper left')
plt.subplot(1, 3, 3)
plt.plot( sub_sks[exi], sub_mzp[exi], 'o', ms=2, alpha=0.1, color='brown', label='ELAIS')
plt.xlabel('i-band skysig')
plt.ylabel('i-band mag_zp')
plt.show()

plt.figure(figsize=(12, 3))
plt.subplot(1, 3, 1)
plt.plot( sub_mjd[cxg], sub_mzp[cxg], 'o', ms=2, alpha=0.1, color='darkgreen', label='COSMOS')
plt.plot( sub_mjd[exg], sub_mzp[exg], 'o', ms=2, alpha=0.1, color='limegreen', label='ELAIS')
plt.xlabel('g-band MJD')
plt.ylabel('g-band mag_zp')
# plt.legend(loc='upper left')
plt.subplot(1, 3, 2)
plt.plot( sub_mjd[cxr], sub_mzp[cxr], 'o', ms=2, alpha=0.1, color='red', label='COSMOS')
plt.plot( sub_mjd[exr], sub_mzp[exr], 'o', ms=2, alpha=0.1, color='darkorange', label='ELAIS')
plt.xlabel('r-band MJD')
plt.ylabel('r-band mag_zp')
# plt.legend(loc='upper left')
plt.subplot(1, 3, 3)
plt.plot( sub_mjd[exi], sub_mzp[exi], 'o', ms=2, alpha=0.1, color='brown', label='ELAIS')
plt.xlabel('i-band MJD')
plt.ylabel('i-band mag_zp')
plt.show()

In [ ]:
del cxg,cxr,exg,exr,exi

### 6 For which types of subtractions/exposures in g- and r-band is the zeropoint discrepant?

Take a look at all the subtractions, not just those with standard exposure times.

In [ ]:
### COSMOS subtractions in g and r
cxg = np.where( (sub_ra>145) & (sub_ra<155)  & (sub_fil == 'g') )[0]
cxr = np.where( (sub_ra>145) & (sub_ra<155)  & (sub_fil == 'r') )[0]

### ELAIS subtractions in g and r
exg = np.where( (sub_ra>2.0) & (sub_ra<12.0) & (sub_fil == 'g') )[0]
exr = np.where( (sub_ra>2.0) & (sub_ra<12.0) & (sub_fil == 'r') )[0]

In [ ]:
plt.figure(figsize=(12, 3))
plt.subplot(1, 2, 1)
plt.plot( sub_mjd[cxg], sub_mzp[cxg], 'o', ms=2, alpha=0.1, color='darkgreen', label='COSMOS')
plt.plot( sub_mjd[exg], sub_mzp[exg], 'o', ms=2, alpha=0.1, color='limegreen', label='ELAIS')
plt.xlabel('g-band MJD')
plt.ylabel('g-band mag_zp')
# plt.legend(loc='upper left')
plt.subplot(1, 2, 2)
plt.plot( sub_mjd[cxr], sub_mzp[cxr], 'o', ms=2, alpha=0.1, color='red', label='COSMOS')
plt.plot( sub_mjd[exr], sub_mzp[exr], 'o', ms=2, alpha=0.1, color='darkorange', label='ELAIS')
plt.xlabel('r-band MJD')
plt.ylabel('r-band mag_zp')
# plt.legend(loc='upper left')
plt.show()

plt.figure(figsize=(12, 3))
plt.subplot(1, 2, 1)
plt.plot( sub_expt[cxg], sub_mzp[cxg], 'o', ms=2, alpha=0.1, color='darkgreen', label='COSMOS')
plt.plot( sub_expt[exg], sub_mzp[exg], 'o', ms=2, alpha=0.1, color='limegreen', label='ELAIS')
plt.xlabel('g-band exptime')
plt.ylabel('g-band mag_zp')
# plt.legend(loc='upper left')
plt.subplot(1, 2, 2)
plt.plot( sub_expt[cxr], sub_mzp[cxr], 'o', ms=2, alpha=0.1, color='red', label='COSMOS')
plt.plot( sub_expt[exr], sub_mzp[exr], 'o', ms=2, alpha=0.1, color='darkorange', label='ELAIS')
plt.xlabel('r-band exptime')
plt.ylabel('r-band mag_zp')
# plt.legend(loc='upper left')
plt.show()

plt.figure(figsize=(12, 3))
plt.subplot(1, 2, 1)
plt.plot( sub_lmg[cxg], sub_mzp[cxg], 'o', ms=2, alpha=0.1, color='darkgreen', label='COSMOS')
plt.plot( sub_lmg[exg], sub_mzp[exg], 'o', ms=2, alpha=0.1, color='limegreen', label='ELAIS')
plt.xlabel('g-band lmt_mg')
plt.ylabel('g-band mag_zp')
# plt.legend(loc='upper left')
plt.subplot(1, 2, 2)
plt.plot( sub_lmg[cxr], sub_mzp[cxr], 'o', ms=2, alpha=0.1, color='red', label='COSMOS')
plt.plot( sub_lmg[exr], sub_mzp[exr], 'o', ms=2, alpha=0.1, color='darkorange', label='ELAIS')
plt.xlabel('r-band lmt_mg')
plt.ylabel('r-band mag_zp')
# plt.legend(loc='upper left')
plt.show()

plt.figure(figsize=(12, 3))
plt.subplot(1, 2, 1)
plt.plot( sub_see[cxg], sub_mzp[cxg], 'o', ms=2, alpha=0.1, color='darkgreen', label='COSMOS')
plt.plot( sub_see[exg], sub_mzp[exg], 'o', ms=2, alpha=0.1, color='limegreen', label='ELAIS')
plt.xlabel('g-band seeing')
plt.ylabel('g-band mag_zp')
# plt.legend(loc='upper left')
plt.subplot(1, 2, 2)
plt.plot( sub_see[cxr], sub_mzp[cxr], 'o', ms=2, alpha=0.1, color='red', label='COSMOS')
plt.plot( sub_see[exr], sub_mzp[exr], 'o', ms=2, alpha=0.1, color='darkorange', label='ELAIS')
plt.xlabel('r-band seeing')
plt.ylabel('r-band mag_zp')
# plt.legend(loc='upper left')
plt.show()

plt.figure(figsize=(12, 3))
plt.subplot(1, 2, 1)
plt.plot( sub_moonsep[cxg], sub_mzp[cxg], 'o', ms=2, alpha=0.1, color='darkgreen', label='COSMOS')
plt.plot( sub_moonsep[exg], sub_mzp[exg], 'o', ms=2, alpha=0.1, color='limegreen', label='ELAIS')
plt.xlabel('g-band moonsep')
plt.ylabel('g-band mag_zp')
# plt.legend(loc='upper left')
plt.subplot(1, 2, 2)
plt.plot( sub_moonsep[cxr], sub_mzp[cxr], 'o', ms=2, alpha=0.1, color='red', label='COSMOS')
plt.plot( sub_moonsep[exr], sub_mzp[exr], 'o', ms=2, alpha=0.1, color='darkorange', label='ELAIS')
plt.xlabel('r-band moonsep')
plt.ylabel('r-band mag_zp')
# plt.legend(loc='upper left')
plt.show()

plt.figure(figsize=(12, 3))
plt.subplot(1, 2, 1)
plt.plot( sub_moonill[cxg], sub_mzp[cxg], 'o', ms=2, alpha=0.1, color='darkgreen', label='COSMOS')
plt.plot( sub_moonill[exg], sub_mzp[exg], 'o', ms=2, alpha=0.1, color='limegreen', label='ELAIS')
plt.xlabel('g-band moonill')
plt.ylabel('g-band mag_zp')
# plt.legend(loc='upper left')
plt.subplot(1, 2, 2)
plt.plot( sub_moonill[cxr], sub_mzp[cxr], 'o', ms=2, alpha=0.1, color='red', label='COSMOS')
plt.plot( sub_moonill[exr], sub_mzp[exr], 'o', ms=2, alpha=0.1, color='darkorange', label='ELAIS')
plt.xlabel('r-band moonill')
plt.ylabel('r-band mag_zp')
# plt.legend(loc='upper left')
plt.show()

plt.figure(figsize=(12, 3))
plt.subplot(1, 2, 1)
plt.plot( sub_mpixval[cxg], sub_mzp[cxg], 'o', ms=2, alpha=0.1, color='darkgreen', label='COSMOS')
plt.plot( sub_mpixval[exg], sub_mzp[exg], 'o', ms=2, alpha=0.1, color='limegreen', label='ELAIS')
plt.xlim([2000., np.max(sub_mpixval[cxg])])
plt.xlabel('g-band median pixel value [counts]')
plt.ylabel('g-band mag_zp')
# plt.legend(loc='upper left')
plt.subplot(1, 2, 2)
plt.plot( sub_mpixval[cxr], sub_mzp[cxr], 'o', ms=2, alpha=0.1, color='red', label='COSMOS')
plt.plot( sub_mpixval[exr], sub_mzp[exr], 'o', ms=2, alpha=0.1, color='darkorange', label='ELAIS')
plt.xlim([2000., np.max(sub_mpixval[cxr])])
plt.xlabel('r-band median pixel value [counts]')
plt.ylabel('r-band mag_zp')
# plt.legend(loc='upper left')
plt.show()

In [ ]:
del cxg,cxr,exg,exr